In [1]:
import pandas as pd
import pmdarima as pm

In [2]:
df_train = pd.read_csv("META_2022-01-01_2023-09-01.csv", parse_dates=True)
train_close = df_train["Close"]

df_real = pd.read_csv("META_2023-09-01_2023-12-01.csv", parse_dates=True)
real_close_prices = df_real["Close"]
real_open_prices = df_real["Open"]

In [3]:
best_order = (8, 2, 1)
model = pm.ARIMA(order=best_order, suppress_warnings=True)
model = model.fit(train_close)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  418
Model:               SARIMAX(8, 2, 1)   Log Likelihood               -1418.887
Date:                Thu, 21 Dec 2023   AIC                           2859.774
Time:                        14:02:49   BIC                           2904.111
Sample:                             0   HQIC                          2877.305
                                - 418                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0108      0.007      1.637      0.102      -0.002       0.024
ar.L1         -0.0338      0.073     -0.464      0.642      -0.176       0.109
ar.L2         -0.0021      0.060     -0.036      0.972      -0.119       0.115
ar.L3         -0.0726      0.067     -1.085      0.278      -0.204       0.058
ar.L4         -0.1015      0.047     -2.162      0.031      -0.193      -0.009
ar.L5         -0.0409      0.063     -0.645      0.519      -0.165       0.083
ar.L6         -0.0233      0.059     -0.397      0.691      -0.138       0.092
ar.L7          0.0165      0.063      0.262      0.793      -0.107       0.140
ar.L8         -0.0925      0.079     -1.167      0.243      -0.248       0.063
ma.L1         -0.9994      0.204     -4.901      0.000      -1.399      -0.600
sigma2        52.7418     10.240      5.151      0.000      32.672      72.812
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):             23728.05
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.28   Skew:                            -2.80
Prob(H) (two-sided):                  0.00   Kurtosis:                        39.57
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [4]:
results_dict = {
    "Day": [],
    "Action": [],
    "Buy Price": [],
    "Predicted Price": [],
    "Real Price Next Day": [],
    "Profit": [],
    "Portfolio Balance": [],
}

portfolio_balance = 10000

for i in range(len(real_close_prices) - 1):
    current_price = real_close_prices[i]
    next_day_predicted = model.predict(n_periods=1).tolist()[0]
    next_day_real = real_open_prices[i + 1]

    if next_day_predicted >= current_price:
        action = "Buy Long"
        profit = (next_day_real - current_price) / current_price * portfolio_balance
    else:
        action = "Short Sell"
        profit = (current_price - next_day_real) / current_price * portfolio_balance

    portfolio_balance += profit

    model.update(next_day_real)

    results_dict["Day"].append(df_real["Date"][i])
    results_dict["Action"].append(action)
    results_dict["Buy Price"].append(current_price)
    results_dict["Predicted Price"].append(next_day_predicted)
    results_dict["Real Price Next Day"].append(next_day_real)
    results_dict["Profit"].append(profit)
    results_dict["Portfolio Balance"].append(portfolio_balance)

results_df = pd.DataFrame(results_dict)

pd.set_option("display.max_rows", None)
display(results_df)
pd.reset_option("display.max_rows")

,Day,Action,Buy Price,Predicted Price,Real Price Next Day,Profit,Portfolio Balance
0,2023-09-01 00:00:00-04:00,Buy Long,296.380005,297.484776,297.019989,21.593364,10021.593364
1,2023-09-05 00:00:00-04:00,Short Sell,300.149994,297.618897,301.709991,-52.086162,9969.507202
2,2023-09-06 00:00:00-04:00,Buy Long,299.170013,304.179429,298.000000,-38.989393,9930.517809
3,2023-09-07 00:00:00-04:00,Buy Long,298.670013,300.188029,299.220001,18.286615,9948.804424
4,2023-09-08 00:00:00-04:00,Buy Long,297.890015,300.608428,301.410004,117.559101,10066.363525
5,2023-09-11 00:00:00-04:00,Short Sell,307.559998,302.487521,306.329987,40.257959,10106.621484
6,2023-09-12 00:00:00-04:00,Buy Long,301.660004,308.653565,302.359985,23.451733,10130.073217
7,2023-09-13 00:00:00-04:00,Short Sell,305.059998,304.331573,306.739990,-55.787219,10074.285998
8,2023-09-14 00:00:00-04:00,Short Sell,311.720001,308.217026,311.609985,3.555535,10077.841533
9,2023-09-15 00:00:00-04:00,Buy Long,300.309998,312.858221,298.190002,-71.143069,10006.698464
